In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
ANNOTATIONS_FILE = r"D:/CROPPED-CORALS/index.csv"
IMG_DIR = r"D:/CROPPED-CORALS/"

num_classes = 34   # 34 classes detected for initial dataset
num_epochs = 20
batch_size = 16
learning_rate = 0.01

model_verbose = False

In [3]:
from model.resnet import ResNet, ResidualBlock
from model.resnet_hdc import ResNet18_HDC

model = ResNet18_HDC(num_classes=num_classes, verbose=model_verbose).to(device)
print(f"Attempting to run ResNet18 with HDC on device {device}")
print(f"Model architecture: {str(model)}")

Attempting to run ResNet18 with HDC on device cpu
Model architecture: ResNet18_HDC(
  (res1_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (res1_2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (res1_3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (res2_1): ResidualHDCBlock_O(
    (res_1x1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [4]:
from dataset.image_dataset import ImageDataset

def data_loader(batch_size, random_seed=42, valid_size=0.1, shuffle=True, test=False):
    # define transforms
    transform = transforms.ToTensor()
    target_transform = None

    if test:
        dataset = ImageDataset(
            ANNOTATIONS_FILE, 
            IMG_DIR, 
            transform=transform, 
            target_transform=target_transform, 
            random_state=random_seed
        )

        data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=shuffle
        )

        return data_loader

    # load the dataset
    train_dataset = ImageDataset(ANNOTATIONS_FILE, IMG_DIR, train=True, transform=transform, target_transform=target_transform, random_state=random_seed)
    valid_dataset = ImageDataset(ANNOTATIONS_FILE, IMG_DIR, train=True, transform=transform, target_transform=target_transform, random_state=random_seed)

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler)

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler)

    return (train_loader, valid_loader)

In [5]:
train_loader, valid_loader = data_loader(batch_size=batch_size)
test_loader = data_loader(batch_size=batch_size, test=True)

Annotations File: D:/CROPPED-CORALS/index.csv
Image Base Directory: D:/CROPPED-CORALS/
Train=True, Transform=ToTensor(), Target Transform=None, Random State=42
Annotations File: D:/CROPPED-CORALS/index.csv
Image Base Directory: D:/CROPPED-CORALS/
Train=True, Transform=ToTensor(), Target Transform=None, Random State=42
Annotations File: D:/CROPPED-CORALS/index.csv
Image Base Directory: D:/CROPPED-CORALS/
Train=False, Transform=ToTensor(), Target Transform=None, Random State=42


In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)  

# Train the model
import gc
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        print(f"Epoch {epoch + 1}/{num_epochs} | Batch {i}")

        #Move tensors to the configured device
        images = images.float()
        images = images.to(device)
        labels = labels.long()
        labels = labels.to(device)

        #Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()

    print ('Epoch [{}/{}], Loss: {:.4f}' 
                .format(epoch+1, num_epochs, loss.item()))

#Validation
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))


Epoch 1/20 | Batch 0
